In [ ]:
import platform
import subprocess
from ipaddress import ip_address

# BLOCKING:
def ping(host, timeout=5.0):
    """
    Returns True if host (str) responds to a ping request.
    Remember that a host may not respond to a ping (ICMP) request even if the host name is valid.
    """
    # check for validity; ValueError if invalid
    host = ip_address(host)
    
    # Option for the number of packets as a function of
    param = '-n' if platform.system().lower()=='windows' else '-c'

    # Building the command. Ex: "ping -c 1 google.com"
    command = ['ping', param, '1', str(host)]

    try:
        val = subprocess.call(command, timeout=timeout)
        return val == 0
    except subprocess.TimeoutExpired:
        return False

In [ ]:
ping("127.0.0.1", timeout=.2)

In [ ]:
ping("127.0.0.0", timeout=.2)

In [18]:
from subprocess import Popen, PIPE
from ipaddress import ip_address
import platform
import time

# ASYNCHRONOUS AND PARALLEL
class Class:
    
    def ping_ips(self, hosts):
        
        # Check if running, return if so
        
        self.ping_procs = []
        self.ping_values = []
        
        for host in hosts:

            # check for validity; ValueError if invalid
            host = ip_address(host)

            # Option for the number of packets as a function of
            param = '-n' if platform.system().lower()=='windows' else '-c'

            # Building the command. Ex: "ping -c 1 google.com"
            command = ['ping', param, '3', str(host)]

            self.ping_procs += [Popen(command,stdout=PIPE)]
        
        self.ping_values = [None for p in self.ping_procs]
        
    def check_pings(self):
        
        for i, p in enumerate(self.ping_procs):
            
            if p.poll() is not None:
                # Done running, get output
                try:
                    out,err = p.communicate()
                    self.ping_values[i] = str(out,'utf-8')
                    p.kill()
                except ValueError:
                    # p has already been killed
                    pass
                
        return self.ping_values

In [19]:
c = Class()

In [25]:
c.ping_ips(['7.0.0.1', '7.0.0.4', '7.0.0.3'])

In [26]:
c.ping_procs

In [31]:
pings = c.check_pings()

for p in pings:
    print('>>>', p)

>>> PING 7.0.0.1 (7.0.0.1): 56 data bytes
Request timeout for icmp_seq 0
Request timeout for icmp_seq 1

--- 7.0.0.1 ping statistics ---
3 packets transmitted, 0 packets received, 100.0% packet loss

>>> PING 7.0.0.4 (7.0.0.4): 56 data bytes
64 bytes from 7.0.0.4: icmp_seq=0 ttl=128 time=2.357 ms
64 bytes from 7.0.0.4: icmp_seq=1 ttl=128 time=2.414 ms
64 bytes from 7.0.0.4: icmp_seq=2 ttl=128 time=2.975 ms

--- 7.0.0.4 ping statistics ---
3 packets transmitted, 3 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 2.357/2.582/2.975/0.279 ms

>>> PING 7.0.0.3 (7.0.0.3): 56 data bytes
64 bytes from 7.0.0.3: icmp_seq=0 ttl=64 time=635.320 ms
64 bytes from 7.0.0.3: icmp_seq=1 ttl=64 time=176.884 ms
64 bytes from 7.0.0.3: icmp_seq=2 ttl=64 time=64.039 ms

--- 7.0.0.3 ping statistics ---
3 packets transmitted, 3 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 64.039/292.081/635.320/247.040 ms



In [32]:
for p in c.ping_procs:
    p.terminate()

In [81]:
def parse_ping(ping):
    """ macOS ping parsing """
    
    value = "-- ms"

    if ping is None or 'Request timeout' in ping:
        return value

    blob = ping.split('\n')[-2] # "line after ping statistics"
    
    if blob == '':
        return value

    # get avg from roundtrip min/avg/max/stddev line
    value = (blob.split('/')[-3]) + ' ms'
    
    return value

In [66]:
parse_ping(pings[0])

'-- ms'

In [67]:
parse_ping(pings[1])

'2.582 ms'

In [68]:
parse_ping(pings[2])

'292.081 ms'

In [122]:
from subprocess import Popen, PIPE

# ASYNCHRONOUS AND PARALLEL
class ParallelApplication:
    
    def run_commands(self, commands, errors=False):
        
        # Check if running, return if so
        
        self.procs = []
        self.returns = []
        
        for cmd in commands:
            if errors:
                stderr = PIPE
            else:
                stderr = None
                
            self.procs += [Popen(cmd, stdout=PIPE, stderr=stderr)]
        
        self.returns = [None for p in self.procs]
        
    def check_returns(self):
        
        for i, p in enumerate(self.procs):
            
            if p.poll() is not None:
                # Done running, get output
                try:
                    out,err = p.communicate()
                    self.returns[i] = str(out,'utf-8')
                    p.kill()
                    
                except ValueError:
                    # p has already been killed
                    pass
                
        return self.returns

In [130]:
hosts = ['127.0.0.1', '9.0.0.1', "10.0.0.134"]

commands = []

for host in hosts:

    # check for validity; ValueError if invalid
    host = ip_address(host)

    # Option for the number of packets as a function of
    param = '-n' if platform.system().lower()=='windows' else '-c'

    # Building the command. Ex: "ping -c 1 google.com"
    command = ['ping', param, '1', str(host)]
    
    commands += [command]

In [131]:
p = ParallelApplication()
p.run_commands(commands)

In [136]:
for r in p.check_returns():
    print(r)

PING 127.0.0.1 (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.060 ms

--- 127.0.0.1 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.060/0.060/0.060/0.000 ms

PING 9.0.0.2 (9.0.0.2): 56 data bytes

--- 9.0.0.2 ping statistics ---
1 packets transmitted, 0 packets received, 100.0% packet loss

PING 10.0.0.134 (10.0.0.134): 56 data bytes
64 bytes from 10.0.0.134: icmp_seq=0 ttl=128 time=4.363 ms

--- 10.0.0.134 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 4.363/4.363/4.363/0.000 ms



In [140]:
p.check_returns()[0].split('\n')

['PING 127.0.0.1 (127.0.0.1): 56 data bytes',
 '64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.060 ms',
 '',
 '--- 127.0.0.1 ping statistics ---',
 '1 packets transmitted, 1 packets received, 0.0% packet loss',
 'round-trip min/avg/max/stddev = 0.060/0.060/0.060/0.000 ms',
 '']

In [ ]:
!ping -c 1 7.0.0.1

In [70]:
from subprocess import Popen, PIPE

def ping_many(ips, errors=False):
    
    commands = [f"ping -i .5 -t 1 -c 1 {ip}" for ip in ips]

    # Check if running, return if so
    
    request = {'procs': [],'returns': []}

    for cmd in commands:
        
        if errors:
            stderr = PIPE
        else:
            stderr = None

        request['procs'] += [Popen(cmd, stdout=PIPE, stderr=stderr, shell=True)]

    request['returns'] = ['-- ms' for p in request['procs']]
    
    return request

def parse_ping(ping):
    """ macOS ping parsing """
    
    value = "-- ms"
    
    if ping is None:
        return value
    
    blob = ping.split('\n')[1]    

    if blob == '':
        return value
    
    if ' ' in blob:
        blob = blob.split(' ')

        if len(blob) > 3:
            value = blob[-2].split('=')[1] + " ms"
            
    return value

def get_pings(request):

    for i, p in enumerate(request['procs']):

        if p.poll() is not None:
            
            # Done running, get output
            try:
                out,err = p.communicate()
                request['returns'][i] = parse_ping(str(out,'utf-8'))
                p.kill()

            except ValueError:
                # p has already been killed
                pass        

    return request

In [71]:
r = ping_many(['127.0.0.1', '73.162.86.31', '7.0.0.2'])

In [73]:
get_pings(r)

Confirmed


{'procs': [<subprocess.Popen at 0x7fd4f837c990>,
 'returns': ['0.047 ms', '32.806 ms', '-- ms']}

In [74]:
{0:"a", 'b':2}

{0: 'a', 'b': 2}

In [88]:
!ping -i .1 -t 1 -c 3 7.0.0.3

PING 7.0.0.3 (7.0.0.3): 56 data bytes
Request timeout for icmp_seq 0
64 bytes from 7.0.0.3: icmp_seq=0 ttl=64 time=153.472 ms
64 bytes from 7.0.0.3: icmp_seq=1 ttl=64 time=118.156 ms
64 bytes from 7.0.0.3: icmp_seq=2 ttl=64 time=64.202 ms

--- 7.0.0.3 ping statistics ---
3 packets transmitted, 3 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 64.202/111.943/153.472/36.708 ms


In [94]:
start = time.time()

In [95]:
!ping -i 1 -t 9 -c 5 7.0.0.3

PING 7.0.0.3 (7.0.0.3): 56 data bytes
64 bytes from 7.0.0.3: icmp_seq=0 ttl=64 time=100.296 ms
64 bytes from 7.0.0.3: icmp_seq=1 ttl=64 time=93.124 ms
64 bytes from 7.0.0.3: icmp_seq=2 ttl=64 time=121.827 ms
64 bytes from 7.0.0.3: icmp_seq=3 ttl=64 time=171.958 ms
64 bytes from 7.0.0.3: icmp_seq=4 ttl=64 time=78.303 ms

--- 7.0.0.3 ping statistics ---
5 packets transmitted, 5 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 78.303/113.102/171.958/32.601 ms


In [96]:
time.time() - start

4.224144697189331